In [2]:
import csv
from bs4 import BeautifulSoup
from requests import get
import unicodedata
import re
import pandas as pd
from time import sleep

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from time import sleep
import csv

from bs4 import BeautifulSoup
from requests import get

In [4]:
driver = webdriver.Chrome()

In [5]:
url = "https://www.amazon.com/"
driver.get(url)
sleep(2)

In [24]:
def get_url(search_term):
    ### Generate a url from search term"
    template = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_2'
    search_term = search_term.replace(' ','+')
    return template.format(search_term)

url = get_url('ultrawilde monitor')
print(url)

https://www.amazon.com/s?k=ultrawilde+monitor&ref=nb_sb_noss_2


In [25]:
def printed_url(func):
    print(f'To jest wyprinotwna wartosc drugiej funkcji: {func}')

printed_url(url)
    

To jest wyprinotwna wartosc drugiej funkcji: https://www.amazon.com/s?k=ultrawilde+monitor&ref=nb_sb_noss_2


In [29]:
driver.get(url)

## Extract the collection

In [30]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
soup

<html class="a-js a-audio a-video a-canvas a-svg a-drag-drop a-geolocation a-history a-webworker a-autofocus a-input-placeholder a-textarea-placeholder a-local-storage a-gradients a-transform3d a-touch-scrolling a-text-shadow a-text-stroke a-box-shadow a-border-radius a-border-image a-opacity a-transform a-transition a-ember" data-19ax5a9jf="dingo" data-aui-build-date="3.22.1-2022-06-28" lang="en-us"><!-- sp:feature:head-start --><head><script async="" crossorigin="anonymous" src="https://images-na.ssl-images-amazon.com/images/I/31YXrY93hfL.js"></script><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>
<!-- sp:end-feature:head-start -->
<script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>
<!-- sp:feature:cs-optimization -->
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link href="https://images-na.ssl-images-amazon.com" rel="dns-prefetch"/>
<link href="https://m.media-amazon.com" rel="dns-prefetch"/>
<link href="https://comple

In [31]:
results = soup.find_all('div', attrs ={'data-component-type': 's-search-result' })
len(results)

22

## Prototype the results

In [32]:
item = results[0]
title = item.h2.a
description = title.text.strip()
url = 'https://www.amazon.com'+ title.get('href')

In [33]:
price_parent = item.find('span', 'a-price')
price = price_parent.find('span','a-offscreen')
price.text

'$229.99'

In [34]:
rating = item.i.text
rating

'4.6 out of 5 stars'

In [35]:
review_count = item.find('span','a-size-base s-underline-text').text

## Generalize the pattern into function


In [36]:
def extract_record(item):
    """Extract and return data from a single record"""
    
    #description and url
    
    title = item.h2.a
    description = title.text.strip()
    url = 'https://www.amazon.com'+ title.get('href')
    
    try:
        #price
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span','a-offscreen').text
    except AttributeError:
        return
    
    try:
        # rank and rating
        rating = item.i.text
        review_count = item.find('span','a-size-base s-underline-text').text
    except:
        rating = ''
        review_count = ''
        
    result = (description, price, rating, review_count, url)
    
    return result

print(extract_record(results[0]))

('ViewSonic OMNI VX3218-PC-MHD 32 Inch Curved 1080p 1ms 165Hz Gaming Monitor with Adaptive Sync, Eye Care, HDMI and Display Port', '$229.99', '4.6 out of 5 stars', '2,479', 'https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A00842293BKX4T98YAIQ0&url=%2FViewSonic-VX3218-PC-MHD-Monitor-Adaptive-Sync-Frameless%2Fdp%2FB08V85DH1R%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dultrawide%2Bmonitor%26qid%3D1656633021%26sr%3D8-1-spons%26psc%3D1&qualifier=1656633021&id=220403046821443&widgetName=sp_atf')


In [37]:
records = []
results = soup.find_all('div', attrs ={'data-component-type': 's-search-result' })

for item in results:
    record = extract_record(item)
    if record:
        records.append(extract_record(item))

In [39]:
records[0]

('ViewSonic OMNI VX3218-PC-MHD 32 Inch Curved 1080p 1ms 165Hz Gaming Monitor with Adaptive Sync, Eye Care, HDMI and Display Port',
 '$229.99',
 '4.6 out of 5 stars',
 '2,479',
 'https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A00842293BKX4T98YAIQ0&url=%2FViewSonic-VX3218-PC-MHD-Monitor-Adaptive-Sync-Frameless%2Fdp%2FB08V85DH1R%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dultrawide%2Bmonitor%26qid%3D1656633021%26sr%3D8-1-spons%26psc%3D1&qualifier=1656633021&id=220403046821443&widgetName=sp_atf')

In [40]:
for row in records:
    print(row[1])

$229.99
$349.99
$379.99
$249.97
$299.99
$439.99
$414.66
$229.99
$398.00
$396.99
$196.99
$346.99
$549.99
$619.00
$409.99
$799.99
$349.99
$329.99
$795.14


## Navigate to the next page

In [41]:
def get_url(search_term):
    ### Generate a url from search term"
    template = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_2'
    search_term = search_term.replace(' ','+')
    
    #add term query to url
    url = template.format(search_term)
    
    #add page query placeholder
    url += '&page{}'
    
    return url

url_search = get_url('ultrawilde monitor')
print(url_search)

https://www.amazon.com/s?k=ultrawilde+monitor&ref=nb_sb_noss_2&page{}


## Putting all together

In [46]:
def get_url(search_term):
    ### Generate a url from search term"
    template = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_2'
    search_term = search_term.replace(' ','+')
    
    #add term query to url
    url = template.format(search_term)
    
    #add page query placeholder
    url += '&page{}'
    
    return url

def extract_record(item):
    """Extract and return data from a single record"""
    
    #description and url
    
    title = item.h2.a
    description = title.text.strip()
    url = 'https://www.amazon.com'+ title.get('href')
    
    try:
        #price
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span','a-offscreen').text
    except AttributeError:
        return
    
    try:
        # rank and rating
        rating = item.i.text
        review_count = item.find('span','a-size-base s-underline-text').text
    except:
        rating = ''
        review_count = ''
        
    result = (description, price, rating, review_count, url)
    
    return result

def main(search_term):
    
    driver = webdriver.Chrome()
    
    records = []
    url = get_url(search_term)
    
    for page in range(1, 21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', attrs ={'data-component-type': 's-search-result' })
        sleep(3)
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
        
    driver.close()
    
    #save data to csv file
    
    with open('result.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'Review_Counts', 'URL'])
        writer.writerows(records)
    

In [47]:
main('ultrawilde monitor')